In [53]:
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
from selenium import webdriver

import time
from tqdm import tqdm

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore')

driver_pathname = 'C:/Users/sujung/Downloads/chromedriver_win32/chromedriver.exe'
# driver_pathname = 'C:/Users/yonsai/Downloads/chromedriver_win32/chromedriver.exe'
playlist1_filename = 'data/melon_playlist1_230528.csv'
playlist2_filename = 'data/melon_playlist2_230528.csv'
genre_ost_filename = 'data/melon_genre_ost_230527.csv'
genre_filename = 'data/melon_genre.csv'

filename = genre_filename

In [54]:
df_plist1 = pd.read_csv(playlist1_filename)
df_plist1.shape, df_plist1.isna().sum()

((2329, 5),
 plylstSeq    0
 title        0
 tag          0
 like         0
 songIds      0
 dtype: int64)

In [55]:
exist_plylstSeqs = df_plist1.plylstSeq.values

In [56]:
# plylstSeq 만 추출해서 중복이 아닌 것만 가려낸다.
df_ost = pd.read_csv(filename)

In [57]:
df_ost.isna().sum()

songId        0
title         0
artist        0
genre         0
album         0
lyricist      2
composer      2
date          1
img           0
comment       0
like          0
lyric        11
plylstSeq     9
dtype: int64

In [58]:
df_ost.shape

(3845, 13)

In [59]:
plylstSeqs = set(' '.join(df_ost[~df_ost.plylstSeq.isna()]['plylstSeq'].values).split())

In [60]:
# 없는 플레이리스트만 가지고 온다.
cnt = 0
print(len(plylstSeqs))
for plylstSeq in plylstSeqs:
    if plylstSeq in exist_plylstSeqs:
        continue
    cnt += 1
print(cnt)

1034
1034


In [61]:
ply_list = [plylstSeq for plylstSeq in plylstSeqs if plylstSeq not in exist_plylstSeqs]
len(ply_list)

1034

In [44]:
plist1, plist2 = [], []

driver = webdriver.Chrome(driver_pathname)

playlist_url = 'https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq='

for ply in tqdm(ply_list):
    
    driver.get(playlist_url + ply)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # plist1에 들어갈 요소들
    title = soup.select_one('.ellipsis.song_name').text.strip()
    tag = soup.select_one('.tag_list.type03').text.replace('\n', ' ')
    like = int(soup.select_one('.cnt').text.split()[-1].replace(',', '').strip())
    
    trs = soup.select('.service_list_song.d_song_list > table > tbody > tr' )
    song_ids = []
    # plist2에 들어갈 요소들
    for tr in trs:
        
        if tr.select_one('.bullet_icons.age_19') :  # 19금 제외
            continue

        img = tr.select_one('img')['src']

        tmp = tr.select_one('.rank01 > span > a')
        s_id = tmp['href'].split(',')[-1][:-2]
        song_ids.append(s_id)

        s_title = tmp.text.strip()
        s_artist = tr.select_one('.rank02 > span').text

        plist2.append({'plylstSeq': ply, 'songId': s_id, 'title': s_title, 'artist': s_artist, 'img': img})
    
    plist1.append({'plylstSeq': ply, 'title': title, 'tag': tag, 'like':like, 'songIds': ' '.join(song_ids)})

driver.close()

100%|██████████| 677/677 [51:33<00:00,  4.57s/it] 


In [45]:
df1 = pd.DataFrame(plist1)
df2 = pd.DataFrame(plist2)

In [46]:
df1.shape, df1.songIds.nunique()

((677, 5), 674)

In [47]:
df1.isna().sum()

plylstSeq    0
title        0
tag          0
like         0
songIds      0
dtype: int64

In [48]:
df2.isna().sum()

plylstSeq    0
songId       0
title        0
artist       0
img          0
dtype: int64

In [49]:
df1.to_csv('data/genre_playlist1_230528.csv', index=False)
df2.to_csv('data/genre_playlist2_230528.csv', index=False)

In [50]:
df1 = pd.read_csv('data/genre_playlist1_230528.csv')
df2 = pd.read_csv('data/genre_playlist2_230528.csv')
df1.shape, df2.shape

((677, 5), (30878, 5))

In [51]:
df1.isna().sum()

plylstSeq    0
title        0
tag          0
like         0
songIds      0
dtype: int64

In [52]:
df2.isna().sum()

plylstSeq    0
songId       0
title        0
artist       0
img          0
dtype: int64